# Quickstart Guide: Accessing Cube Data

In the Cube web application (https://cube.jax.org) you can search through various elements such as Studies, Assays, and Data Sets. Using the controls along the top of the table you can filter based on properties to narrow your search. From the "Data Sets" page, for example, you can select data to add to your collection. Once a data set is in your collection you can use the `cube_r` package to access and inspect those data.

The commands in this notebook show how to access data sets in your collection. Other elements, such as studies and assays, can be inspected here as well, but they won't have any associated data - only metadata.

In [160]:
# load required libraries
# load all library, it is already installed
library(devtools)
library(httr)
library(jsonlite)
library(stringr)
library(rapportools)
library(htmlwidgets)
library(rlist)
library(data.table)
library(cloudml)
library(readr)
library(logger)
library(cube)
setwd("~/")
log_threshold(INFO)

## Logging in

After running the cell below to log in to the Cube service, you will see a link appear in the response. **You will need to click on the `verification_uri_complete` link to complete the login process.** That will associate this notebook session with your user ID and allow you to pull in data from your collection in https://cube.jax.org/ 

In [161]:
# create a CubeAPI object
cube_api = CubeAPI$new()
# only need to login once per month
# after run, click on "verification_uri_complete" to finish verification
cube_api$login()

INFO [2021-01-12 19:25:25] Version: 0.0.2
INFO [2021-01-12 19:25:25] http://10.105.16.22/metadata-service/
INFO [2021-01-12 19:25:25] verification_uri_complete: https://thejacksonlaboratory.auth0.com/activate?user_code=DBRC-FXNT


## Get the data from your collection

In [162]:
response = cube_api$get_metadata_collection()
json = response_json_to_data(response)
str(json$results)

INFO [2021-01-12 19:25:40] GET: http://10.105.16.22/metadata-service/metadata_repository/collection/
INFO [2021-01-12 19:25:40] status_code: 200


No encoding supplied: defaulting to UTF-8.



'data.frame':	1 obs. of  5 variables:
 $ id              : int 10
 $ collection_items:List of 1
  ..$ :'data.frame':	1 obs. of  7 variables:
  .. ..$ id           : int 352
  .. ..$ collection   :'data.frame':	1 obs. of  4 variables:
  .. .. ..$ id             : int 10
  .. .. ..$ collection_name: logi NA
  .. .. ..$ user_name      : chr "waad.939JXKnVNUKLJTw6JEVLxgIlZoYOeIfrJ58NLiCbVQc"
  .. .. ..$ date_created   : chr "2020-12-22T14:24:16.893718Z"
  .. ..$ collection_id: int 10
  .. ..$ accession_id : chr "JAXDS0000G"
  .. ..$ item_type    : chr "Data Set"
  .. ..$ item_label   : logi NA
  .. ..$ date_created : chr "2021-01-12T19:24:47.916557Z"
 $ user_name       : chr "waad.939JXKnVNUKLJTw6JEVLxgIlZoYOeIfrJ58NLiCbVQc"
 $ collection_name : logi NA
 $ date_created    : chr "2020-12-22T14:24:16.893718Z"


## With the accession ID, get the pointer to the data

In [163]:
accession_ids = cube_api$parse_accession_ids(response)
response = cube_api$get_element_instance(accession_ids = c(accession_ids))

INFO [2021-01-12 19:25:50] GET: http://10.105.16.22/metadata-service/metadata_repository/element_instance/
INFO [2021-01-12 19:25:51] status_code: 200


In [157]:
uri = cube_api$parse_storage_uri(response)[1,]
uri

,accession_id,uri,bucket_name,file_name
,<chr>,<chr>,<chr>,<chr>
1,JAXDS0000G,gs://jax-cube-prd-ctrl-01-project-data/study_3-1/Three_bears_CBA_BODY_WEIGHT_200723.txt,gs://jax-cube-prd-ctrl-01-project-data/study_3-1,Three_bears_CBA_BODY_WEIGHT_200723.txt


## Get the data file and read into a data frame

In [164]:
bucket_name = uri[[3]][[1]]
file_name = uri[[4]][[1]]
data_dir = gs_data_dir( bucket_name )
df <- read.table(file.path(data_dir, file_name), sep = '\t',header = TRUE)
head(df)

,X,Barcode,JAX_ASSAY_BLINDID,JAX_ASSAY_PENBARCODE,JAX_ASSAY_PRIMARYID,JAX_ASSAY_PRIMARYIDVALUE,JAX_ASSAY_SECONDAYID,JAX_ASSAY_SECONDARYIDVALUE,JAX_ASSAY_SEX,JAX_ASSAY_COATCOLOR,⋯,EXPERIMENT_SAMPLE.EXPERIMENT.EXPERIMENT_TESTER.JAX_EMPLOYEE_ACTIVEWEBONLYACCOUNT,EXPERIMENT_SAMPLE.EXPERIMENT.EXPERIMENT_TESTER.CI_USERNAME,EXPERIMENT_SAMPLE.EXPERIMENT.EXPERIMENT_TESTER.STATUS,EXPERIMENT_SAMPLE.EXPERIMENT.EXPERIMENT_TESTER.ENFORCE_PROJECT_SECURITY,CLIMB.ID,sex,line,strain,diet,treatment
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<lgl>,<chr>,<chr>,⋯,<lgl>,<chr>,<lgl>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,0,XWE563,3B0101,PN8712,Ear Notch,R,NA,NA,F,black (non-agouti),⋯,FALSE,zoe.sifnakis@jax.org,NA,Yes,3693,Female,JR000664,C57BL/6J,10% fat + fiber,None
2,1,XWE564,3B0112,PN8713,Ear Notch,R,NA,NA,F,agouti,⋯,FALSE,zoe.sifnakis@jax.org,NA,Yes,3773,Female,JR002105,NZO/HILtJ,10% fat + fiber,None
3,2,XWE565,3B0103,PN8714,Ear Notch,R,NA,NA,M,agouti,⋯,FALSE,zoe.sifnakis@jax.org,NA,Yes,3758,Male,JR000928,CAST/EiJ,44% fat+ fiber,None
4,3,XWE566,3B0104,PN8715,Ear Notch,2B,NA,NA,M,agouti,⋯,FALSE,zoe.sifnakis@jax.org,NA,Yes,3796,Male,JR002105,NZO/HILtJ,44% fat+ fiber,None
5,4,XWE567,3B0105,PN8716,Ear Notch,R,NA,NA,F,agouti,⋯,FALSE,zoe.sifnakis@jax.org,NA,Yes,3738,Female,JR000928,CAST/EiJ,44% fat+ fiber,None
6,5,XWE568,3B0106,PN8717,Ear Notch,L,NA,NA,F,agouti,⋯,FALSE,zoe.sifnakis@jax.org,NA,Yes,3774,Female,JR002105,NZO/HILtJ,10% fat + fiber,None


In [165]:
df[c('CLIMB.ID','sex','line','strain','diet','treatment',"JAX_ASSAY_BODYWEIGHT")]

CLIMB.ID,sex,line,strain,diet,treatment,JAX_ASSAY_BODYWEIGHT
<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
3693,Female,JR000664,C57BL/6J,10% fat + fiber,None,20.18
3773,Female,JR002105,NZO/HILtJ,10% fat + fiber,None,35.91
3758,Male,JR000928,CAST/EiJ,44% fat+ fiber,None,14.97
3796,Male,JR002105,NZO/HILtJ,44% fat+ fiber,None,57.22
3738,Female,JR000928,CAST/EiJ,44% fat+ fiber,None,12.23
3774,Female,JR002105,NZO/HILtJ,10% fat + fiber,None,33.48
3713,Male,JR000664,C57BL/6J,10% fat + fiber,None,27.37
3778,Female,JR002105,NZO/HILtJ,44% fat+ fiber,None,45.58
3790,Male,JR002105,NZO/HILtJ,10% fat + fiber,None,47.86
